In [21]:
# Dependencies
import requests
import json
import pandas as pd
from pprint import pprint
from config import api_key

In [27]:
# Specify the URL
elections_url = "https://api.open.fec.gov/v1/elections/?"

# Set params in URL
sort_null_only="true"
office="senate"
state="CA"
per_page="34"
sort_nulls_last="true"
sort_hide_null="true"
cycle="2016"
sort="-total_receipts"
hide_null="true"
page=1
election_full="true"

In [28]:
# Define params
params= {
    "sort_null_only":sort_null_only,
    "office":office,
    "state":state,
    "per_page":per_page,
    "sort_nulls_last":sort_nulls_last,
    "sort_hide_null":sort_hide_null,
    "cycle":cycle,
    "sort":sort,
    "hide_null":hide_null,
    "page":page,
    "election_full":election_full,
    "api_key":api_key
}

# Run request in JSON
response=requests.get(elections_url,params=params).json()

In [48]:
# Pprint the response
pprint(response)

{'api_version': '1.0',
 'pagination': {'count': 33, 'page': 1, 'pages': 1, 'per_page': 34},
 'results': [{'candidate_election_year': 2016,
              'candidate_id': 'S6CA00584',
              'candidate_name': 'HARRIS, KAMALA D',
              'candidate_pcc_id': 'C00571919',
              'candidate_pcc_name': 'KAMALA HARRIS FOR SENATE',
              'cash_on_hand_end_period': 928885.0,
              'committee_ids': ['C00571919'],
              'coverage_end_date': '2016-12-31T00:00:00',
              'incumbent_challenge_full': 'Open seat',
              'party_full': 'DEMOCRATIC PARTY',
              'total_disbursements': 14420725.0,
              'total_receipts': 15349610.0},
             {'candidate_election_year': 2016,
              'candidate_id': 'S6CA00691',
              'candidate_name': 'SANCHEZ, LORETTA',
              'candidate_pcc_id': 'C00578344',
              'candidate_pcc_name': 'LORETTA SANCHEZ FOR SENATE',
              'cash_on_hand_end_period': 83842.0

In [43]:
# Set variable to represent only results from the API
results=response["results"]

# Create empty lists
committee_ids=[]
cash_on_hand_end_period=[]
candidate_pcc_id=[]
total_receipts=[]
coverage_end_date=[]
candidate_election_year=[]
candidate_name=[]
total_disbursements=[]
party_full=[]
candidate_id=[]

In [46]:
# Loop through every result in results
for result in results:
    
    # Try to add information from results to the empty list
    try:
        committee_ids.append(result["committee_ids"][0])
        cash_on_hand_end_period.append(result["cash_on_hand_end_period"])
        candidate_pcc_id.append(result["candidate_pcc_id"])
        total_receipts.append(result["total_receipts"])
        coverage_end_date.append(result["coverage_end_date"])
        candidate_election_year.append(result["candidate_election_year"])
        candidate_name.append(result["candidate_name"])
        total_disbursements.append(result["total_disbursements"])
        party_full.append(result["party_full"])
        candidate_id.append(result["candidate_id"])
        
    # Otherwise return none
    except:
        None
      

In [49]:
# Create a dataframe to show all results
election_df = pd.DataFrame({
    "Committee ID": committee_ids,
    "Cash on Hand End Period": cash_on_hand_end_period,
    "Candidate PCC ID":candidate_pcc_id,
    "Total Receipts": total_receipts,
    "End Date": coverage_end_date,
    "Election Year": candidate_election_year,
    "Candidate Name": candidate_name,
    "Total Disbursements": total_disbursements,
    "Party": party_full,
    "Candidate ID": candidate_id
})

# Show dataframe
election_df

,Committee ID,Cash on Hand End Period,Candidate PCC ID,Total Receipts,End Date,Election Year,Candidate Name,Total Disbursements,Party,Candidate ID
0,C00571919,928885.00,C00571919,15349610.00,2016-12-31T00:00:00,2016,"HARRIS, KAMALA D",14420725.0,DEMOCRATIC PARTY,S6CA00584
1,C00578344,83842.00,C00578344,4566565.00,2016-12-31T00:00:00,2016,"SANCHEZ, LORETTA",4482978.0,DEMOCRATIC PARTY,S6CA00691
2,C00587154,0.00,C00587154,1002704.00,2016-06-30T00:00:00,2016,"SUNDHEIM, GEORGE 'DUF'",1002702.0,REPUBLICAN PARTY,S6CA00741
3,C00573238,0.00,C00573238,470923.00,2016-12-21T00:00:00,2016,"DEL BECCARO, THOMAS",524810.0,REPUBLICAN PARTY,S6CA00683
4,C00573832,0.00,C00573832,119881.00,2016-12-31T00:00:00,2016,"CHAVEZ, ROCKY",123171.0,REPUBLICAN PARTY,S6CA00626
5,C00600742,0.00,C00600742,71366.00,2016-06-30T00:00:00,2016,"GREY, CLIVE",71366.0,INDEPENDENT,S6CA00774
6,C00614537,0.00,C00614537,69151.00,2016-07-27T00:00:00,2016,"UNZ, RON",69149.0,REPUBLICAN PARTY,S8CA00291
7,C00612093,94.00,C00612093,51715.00,2016-12-31T00:00:00,2016,"CONLON, GREG",51621.0,REPUBLICAN PARTY,S2CA00740
8,C00575944,30737.54,C00575944,48900.00,2015-12-31T00:00:00,2016,"WYMAN, PHILLIP D",11761.0,REPUBLICAN PARTY,S6CA00667
9,C00573220,0.00,C00573220,30949.00,2015-06-30T00:00:00,2016,"ALBERTSON, STEWART",30949.0,DEMOCRATIC PARTY,S6CA00618
